In [ ]:
%xmode Verbose

In [ ]:
## install stuff
import subprocess
command = ["pip3", "install", "urllib"]
a = ["PyPDF2", "bs4"]
for i in a:
  command.append(i)
  subprocess.run(command, capture_output=True)
  command = ["pip3", "install"]

In [ ]:
import PyPDF2
import requests 
import re
from bs4 import BeautifulSoup
import os
import urllib.request
from PyPDF2 import PdfFileWriter, PdfFileReader
import datetime


# scraping using regular expression 
url1 = "https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=medguide.page"
page1 = requests.get(url1)
soup1 = BeautifulSoup(page1.content, 'html.parser')
urlregex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
urls = re.findall(urlregex,soup1.prettify())     
lst = []
for i in urls:
  if len(i)>0:
    if str(i).find("pdf")!=-1:
      for j in i:
        if len(j) > 0:
          lst.append(j)

lst = lst[2:]


#!wget https://www.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png



In [ ]:
#delete files


import subprocess
for i in os.listdir():
  if str(i).find("pdf") != -1:
     subprocess.run(["rm", str(i)])




def handle_a_link(name, link):
  if str(link).find("page") !=-1:
    regex= r"page=\d+"
    num = re.findall(regex , str(link))[0]
    num = re.findall("\d+" , num)[0]
    try:
      urllib.request.urlretrieve(link, "temp.pdf" )
      inputpdf = PdfFileReader(open("temp.pdf", "rb"))
      if int(num) > inputpdf.pages.lengthFunction():
         print("FDA mistake detected", link)
      else:
        myrange = (num, inputpdf.pages.lengthFunction())
        output = PdfFileWriter()
        for i in range(int(myrange[0])-1,int(myrange[1])):
            output.addPage(inputpdf.getPage(i))
        medguidename = name
        with open("%s.pdf" % medguidename, "wb") as outputStream:
          output.write(outputStream)
    except:
      pass
  else:
      medguidename = name
      try:
        urllib.request.urlretrieve(link, "%s.pdf" % medguidename)
      except:
        pass


import PyPDF2
import requests 
import re
from bs4 import BeautifulSoup

url1 = "https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=medguide.page"

page1 = requests.get(url1)
soup1 = BeautifulSoup(page1.content, 'html.parser')




tab = soup1.find_all("td")
len(tab)
block= len(tab)/7
dicto= {}

#for i in range(100):
for i in range(int(block)):
  currentplace = i*7

  drug_name = 0 + currentplace
  active = 1 + currentplace
  form = 2 + currentplace
  Company = 4 + currentplace
  link = 6 + currentplace

  drug_name = tab[drug_name].text[1:]
  drug_name = re.match("\S+", drug_name).group()
  active = tab[active].text
  active = re.match("\S+", active).group()
  form = tab[form].text
  #form = re.match("\S+", form).group()
  Company = tab[Company].text
  Company = re.match("\S+", Company).group()
  link = tab[link].text
  link = re.match("\S+", link).group()
  medguidename = str(drug_name) +" "+ str(active) +" "+ str(form)+" "+str(Company) + " MedGuide"
  medguidename = medguidename.replace("/", " ")
  dicto[medguidename] = link

print(datetime.datetime.now())
for name, link in dicto.items():
  handle_a_link(name, link)
print(datetime.datetime.now())


for i in os.listdir():
  if str(i).find("pdf") != -1:
    subprocess.run(["mv", str(i), "mydir/" +str(i) ])

subprocess.run(["rm", "mydir/temp.pdf"])
subprocess.run(["zip", "-r","MedGuides.zip", "mydir"])
files.download("MedGuides.zip")
